**Setup**

Installs

In [ ]:
!pip install swig
!pip install "stable-baselines3[extra]>=2.0.0a4"
!pip install box2d-py
!apt-get update && apt-get install ffmpeg freeglut3-dev xvfb  # For visualization

Function to download Google Drive files to notebook's filesystem, so they can be easily accessed

In [2]:
import requests

def download_drive_file(file_name, path):
  request = requests.get(path)
  with open(file_name, "wb") as file:
    file.write(request.content)

Create folder to hold example videos (and json files)

In [4]:
import os

example_videos_path = "videos"
if os.path.isdir(example_videos_path) == False:
  os.mkdir(example_videos_path)

Downloads my DQN model and example episodes video

In [5]:
from genericpath import isfile
import os

model_file_name = "dqn_lunarlander.zip"
model_id = "1q9KU8-eWSwOjtmX9h4grCFIry7usrupT"
if os.path.isfile(model_file_name) == False:
  download_drive_file(model_file_name, f"https://drive.google.com/uc?export=download&id={model_id}")

example_video_file_name = "ag-example-lunarlander-video-0-5000.mp4"
example_video_id = "1i4Lj9-_SThtRyNwHTsr43dS6MpcOLiOf"
file_path = os.path.join(example_videos_path, example_video_file_name)
if os.path.isfile(file_path) == False:
  download_drive_file(file_path, f"https://drive.google.com/uc?export=download&id={example_video_id}")

example_json_file_name = "ag-example-lunarlander-json-0-5000.json"
example_json_id = "1-W5FRZONrwT0OrM2CE6Yjk7A2V-bbe9w"
file_path = os.path.join(example_videos_path, example_json_file_name)
if os.path.isfile(file_path) == False:
  download_drive_file(file_path, f"https://drive.google.com/uc?export=download&id={example_json_id}")

**Test**

Create Test Environment

In [6]:
import gymnasium as gym

env = gym.make("LunarLander-v2", render_mode="rgb_array")

Load My Model

In [ ]:
from stable_baselines3 import DQN

model = DQN.load("dqn_lunarlander", env=env)

Evaluation

In [ ]:
import numpy as np

def evaluate(model, num_episodes=100, deterministic=True):
    vec_env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = vec_env.reset()
        while not done:
            action, _ = model.predict(obs, deterministic=deterministic)
            obs, reward, done, info = vec_env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))
        # print(i) # Comment this back in to track the evaluation progress

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Num Episodes:", num_episodes)
    print("Mean Reward:", mean_episode_reward)

    return mean_episode_reward

In [ ]:
evaluate(model)

**Test Examples**

Display

In [8]:
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

`show_videos` is a function that plays the video(s)

In [9]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay


def show_videos(video_path="", prefix=""):
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

Display my examples video

In [ ]:
show_videos("videos", prefix="ag")

`record_video` is a function that runs the model and records the output as a video

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    eval_env = DummyVecEnv([lambda: gym.make("LunarLander-v2", render_mode="rgb_array")])
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    eval_env.close()

Record and display your own example episodes, feel free to change the `video_length`.

In [ ]:
prefix = "new-example"
record_video("lunarlander-v2", model, video_length=1500, prefix=prefix)
show_videos("videos", prefix=prefix)